In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

In [ ]:
MY_BUCKET="restaurant_master"
! mkdir -p gcs

In [ ]:
! gcsfuse --implicit-dirs \
--rename-dir-limit=100 \
--max-conns-per-host=100 \
{MY_BUCKET} /content/gcs

I0713 11:07:42.919634 2023/07/13 11:07:42.919594 Start gcsfuse/1.0.0 (Go version go1.20.4) for app "" using mount point: /content/gcs


# ジオコーディングAPIを導入
Geocoding API

In [ ]:
! pip install -U googlemaps

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40718 sha256=371d1174867ee6f7076bc32622fe68b8246ffa92ff9fb4d82bb1dd49ddd80f82
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
import googlemaps
from datetime import datetime
# GCP consoleから取得
gmaps = googlemaps.Client(key='xxx')
# Geocoding an address
geocode_result = gmaps.geocode('東京都港区南麻布5丁目2番32号興和広尾ビル')
pd.json_normalize(geocode_result)
print(geocode_result[0]["geometry"]["location"]["lat"])
print(geocode_result[0]["geometry"]["location"]["lng"])

35.654666
139.7231595


# pipeline

In [ ]:
def address2lat(address):
    geocode_result = gmaps.geocode(address)
    return geocode_result[0]["geometry"]["location"]["lat"]


def address2lng(address):
    geocode_result = gmaps.geocode(address)
    return geocode_result[0]["geometry"]["location"]["lng"]

In [ ]:
xlsx_file = "/content/gcs/restaurant_master.xlsx"

In [ ]:
import pandas as pd

# Excelファイルを読み込み、特定のシートにアクセスする
df = pd.read_excel(xlsx_file)
# データフレームの内容を表示する
df["lat"] = df["address"].apply(address2lat)
df["lng"] = df["address"].apply(address2lng)

# geopandasを導入する

In [ ]:
!pip install geopandas

In [ ]:
df

,id,restaurant,address,tel,open_time,close_time,website,lat,lng
0,1,タイガー餃子軒 広尾,東京都渋谷区広尾5-14-2,03-6277-4051,1130.0,1500.0,https://kiwa-group.co.jp/tiger_hiroo/,35.650402,139.721527
1,2,BURGER MANIA,東京都港区南麻布5-15-25六幸館2F,03-5422-7899,1100.0,2200.0,https://maniacorporation.com/burger-mania,35.651877,139.722454
2,3,いびさ,東京都渋谷区広尾5-16-16エクセル代田ビル2F,03-5421-0570,NaN,NaN,https://tabelog.com/tokyo/A1307/A130703/13106422/,35.649788,139.720599
3,4,PIZZA SALVATORE CUOMO 西麻布,東京都港区西麻布4-10-5 ネクスト西麻布1F,03-5464-8265,1130.0,1430.0,https://www.salvatore.jp/restaurant/nishiazabu/,35.657410,139.723013
4,5,凛と,東京都渋谷区広尾5‐10‐3 フロストバード広尾 1F,050-5494-7959,1130.0,1400.0,https://gd5s411.gorp.jp/,35.649513,139.721042
5,6,Soup Stock Tokyo,東京都港区南麻布5-15-12 美濃屋ビル 1F,03-5447-6080,1100.0,2100.0,https://www.soup-stock-tokyo.com/store/detail/...,35.651175,139.723073
6,7,mosh Grab'nGo,東京都港区南麻布5-16-10,NaN,1000.0,2100.0,https://www.mos.jp/mosh/,35.651255,139.723273
7,8,西麻布 権八,東京都港区西麻布1-13-11,050-5443-1691,1130.0,2200.0,https://gonpachi.jp/nishi-azabu/,35.660207,139.723594
8,9,まんみ 西麻布,東京都港区西麻布4-5-8 LA 西麻布ビル 3F,03-5485-2007,1130.0,1430.0,https://manmi.jp/,35.656527,139.723198
9,10,広尾のメキシコ料理店\nLA JOLLA（ラ・ホイヤ）,東京都渋谷区広尾5-16-3 小安ビル2F,03-3442-1865,1130.0,1530.0,https://www.la-jolla.jp/,35.650134,139.720768


In [ ]:
import geopandas as gpd
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lng'], df['lat']))

In [ ]:
gdf.to_file("point.geojson", driver="GeoJSON")